# LIBRARY

In [40]:
!pip install pmdarima
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# STEP 1

## CORETAN

### PREPROS DATA

In [ ]:
df_all_data = pd.read_csv('DFALL_130522.csv')
mtn = 1000017
df_all = df_all_data[df_all_data['EMATN'] == mtn].reset_index()

df_curr = pd.read_csv('Data Historis USD_IDR.csv')
df_curr['Terakhir'] = df_curr['Terakhir'].str[:-2].str.replace('.','').astype(int)
df_curr = df_curr.loc[:,['Tanggal','Terakhir']]

df_all['BEDAT'] = df_all['BEDAT'].astype(str)
df_all['BEDAT'] = df_all['BEDAT'].str[6:8] + "/" + df_all['BEDAT'].str[4:6] + "/" + df_all['BEDAT'].str[:4]

for i in range(len(df_all['BEDAT'])):
    for j in range(len(df_curr['Tanggal'])):
        if (df_all.loc[i,'BEDAT'] == df_curr.loc[j,'Tanggal']) and (df_all.loc[i,'WAERS'] == 'IDR'):
            idr_to_usd = df_all.loc[i,'UNITPRICE']/df_curr.loc[j,'Terakhir']
            df_all.loc[i, 'UNITPRICE'] = idr_to_usd
            df_all.loc[i, 'WAERS'] = 'USD'
            
df_all = df_all.drop(df_all[df_all['WAERS']=='IDR'].index).reset_index(drop=True)
df_all = df_all.drop_duplicates(subset='BEDAT',keep='last').reset_index(drop=True)

df_all['BEDAT'] = pd.to_datetime(df_all['BEDAT'])

# remove outlier
from scipy import stats
df_all=df_all[(np.abs(stats.zscore(df_all['UNITPRICE'])) < 3)]

# df_all

# DATETIME TO MONTH
df_all_m = df_all.loc[:,['BEDAT','UNITPRICE']].copy()
df_all_m = df_all_m.groupby(pd.Grouper(key='BEDAT', axis=0, 
                        freq='M')).mean().dropna().reset_index()
df_all_m['BEDAT'] = df_all_m['BEDAT'].dt.to_period('M')

df_all_m

### CEK KORELASI

In [ ]:
df_fertecon = pd.read_excel('Data Fertecon 2019 dst.xlsx',sheet_name=1)
df_fertecon.rename(columns={'date':'BEDAT'},inplace=True)
df_fertecon['BEDAT'] = df_fertecon['BEDAT'].str[:4] + '-' + df_fertecon['BEDAT'].str[5:]

df_fertecon['BEDAT'] = pd.to_datetime(df_fertecon['BEDAT'])
df_fertecon = df_fertecon.set_index('BEDAT').to_period('M').reset_index()

for i in range(1,len(df_fertecon.columns)):
  df_fertecon['{}'.format(df_fertecon.columns[i])] = df_fertecon['{}'.format(df_fertecon.columns[i])].astype(float)


# df_fertecon

region_corr_fix = df_all_m.merge(df_fertecon, on='BEDAT', how='left').dropna().corr()['UNITPRICE'].sort_values().index[-2]
region_corr_fix

### GABUNGIN SAP + FERTECON

In [ ]:
tambahan_row = df_fertecon[df_fertecon['BEDAT'] > df_all_m['BEDAT'].iloc[-1]].loc[:,['BEDAT','{}'.format(region_corr_fix)]].rename(columns={'{}'.format(region_corr_fix) : 'UNITPRICE'})
final_data_concat = pd.concat([df_all_m,tambahan_row])
final_data_concat['EMATN'] = mtn
final_data_concat['CORR_BENCHMARK'] = region_corr_fix
final_data_concat

## MAKE FUNCTION STEP 1

In [41]:
df_all_data = pd.read_csv('DFALL_130522.csv')
df_all_data

,BANFN,BNFPO,EBELN,EBELP,BEDAT,EKORG,EKGRP,EMATN,TXZ01,UNITPRICE,CURRENCY_RATE,WAERS,QTY
0,2100001642,10,5000001438,10,20210920,A000,A01,3000019,KALIUM CHLORIDE:KCL;FLAKE,591.0,14.23301,USD,5000.000
1,2200000473,10,5100000823,10,20160413,B000,B01,3000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.0,13.12300,USD,25000.000
2,2200000475,10,5100000837,10,20160414,B000,B01,3000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.0,13.23800,USD,25000.000
3,2200001384,10,5100001812,10,20160427,B000,B01,3000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.0,13.21500,USD,25000.000
4,2200001937,10,5100002059,10,20160504,B000,B01,1000017,DAP:(NH4)2HPO4;GRANULAR;18%N 46%P2O5,344.0,13.24600,USD,13500.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,2600033074,10,5500027840,10,20211203,F000,F01,3000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.0,14.35301,USD,4800.010
433,2600033090,10,5500027838,10,20211203,F000,F01,3000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.0,14.35301,USD,1831.937
434,2600033091,10,5500027837,10,20211203,F000,F01,3000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.0,14.35301,USD,4700.500
435,2600033520,10,5500028311,10,20220125,F000,F01,3000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.0,14.34701,USD,4850.380


In [42]:
df_all_data[df_all_data['WAERS'] == 'USD']

,BANFN,BNFPO,EBELN,EBELP,BEDAT,EKORG,EKGRP,EMATN,TXZ01,UNITPRICE,CURRENCY_RATE,WAERS,QTY
0,2100001642,10,5000001438,10,20210920,A000,A01,3000019,KALIUM CHLORIDE:KCL;FLAKE,591.0,14.23301,USD,5000.000
1,2200000473,10,5100000823,10,20160413,B000,B01,3000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.0,13.12300,USD,25000.000
2,2200000475,10,5100000837,10,20160414,B000,B01,3000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.0,13.23800,USD,25000.000
3,2200001384,10,5100001812,10,20160427,B000,B01,3000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.0,13.21500,USD,25000.000
4,2200001937,10,5100002059,10,20160504,B000,B01,1000017,DAP:(NH4)2HPO4;GRANULAR;18%N 46%P2O5,344.0,13.24600,USD,13500.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,2600033074,10,5500027840,10,20211203,F000,F01,3000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.0,14.35301,USD,4800.010
433,2600033090,10,5500027838,10,20211203,F000,F01,3000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.0,14.35301,USD,1831.937
434,2600033091,10,5500027837,10,20211203,F000,F01,3000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.0,14.35301,USD,4700.500
435,2600033520,10,5500028311,10,20220125,F000,F01,3000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.0,14.34701,USD,4850.380


In [43]:
def olahData(mtn):
  # PREPROS DATA
  df_all_data = pd.read_csv('DFALL_130522.csv')
  df_curr = pd.read_csv('Data Historis USD_IDR.csv')
  df_fertecon = pd.read_excel('Data Fertecon 2019 dst.xlsx',sheet_name=1)

  mtn = mtn
  df_all = df_all_data[df_all_data['EMATN'] == mtn].reset_index()

  
  df_curr['Terakhir'] = df_curr['Terakhir'].str[:-2].str.replace('.','').astype(int)
  df_curr = df_curr.loc[:,['Tanggal','Terakhir']]

  df_all['BEDAT'] = df_all['BEDAT'].astype(str)
  df_all['BEDAT'] = df_all['BEDAT'].str[6:8] + "/" + df_all['BEDAT'].str[4:6] + "/" + df_all['BEDAT'].str[:4]

  for i in range(len(df_all['BEDAT'])):
      for j in range(len(df_curr['Tanggal'])):
          if (df_all.loc[i,'BEDAT'] == df_curr.loc[j,'Tanggal']) and (df_all.loc[i,'WAERS'] == 'IDR'):
              idr_to_usd = df_all.loc[i,'UNITPRICE']/df_curr.loc[j,'Terakhir']
              df_all.loc[i, 'UNITPRICE'] = idr_to_usd
              df_all.loc[i, 'WAERS'] = 'USD'
              
  df_all = df_all.drop(df_all[df_all['WAERS']=='IDR'].index).reset_index(drop=True)
  df_all = df_all.drop_duplicates(subset='BEDAT',keep='last').reset_index(drop=True)

  df_all['BEDAT'] = pd.to_datetime(df_all['BEDAT'])

  ## remove outlier
  from scipy import stats
  df_all=df_all[(np.abs(stats.zscore(df_all['UNITPRICE'])) < 3)]


  ## DATETIME TO MONTH
  df_all_m = df_all.loc[:,['BEDAT','UNITPRICE']].copy()
  df_all_m = df_all_m.groupby(pd.Grouper(key='BEDAT', axis=0, 
                          freq='M')).mean().dropna().reset_index()
  df_all_m['BEDAT'] = df_all_m['BEDAT'].dt.to_period('M')

  if ((mtn == 1000017) | (mtn == 3000499) | (mtn == 3000266) | (mtn == 3000539) | (mtn == 3000508)):
    # MENEMUKAN REGION KORELASI DENGAN FERTECON
    df_fertecon.rename(columns={'date':'BEDAT'},inplace=True)
    df_fertecon['BEDAT'] = df_fertecon['BEDAT'].str[:4] + '-' + df_fertecon['BEDAT'].str[5:]

    df_fertecon['BEDAT'] = pd.to_datetime(df_fertecon['BEDAT'])
    df_fertecon = df_fertecon.set_index('BEDAT').to_period('M').reset_index()

    for i in range(1,len(df_fertecon.columns)):
      df_fertecon['{}'.format(df_fertecon.columns[i])] = df_fertecon['{}'.format(df_fertecon.columns[i])].astype(float)

    region_corr_fix = df_all_m.merge(df_fertecon, on='BEDAT', how='left').dropna().corr()['UNITPRICE'].sort_values().index[-2]
    region_corr_fix

    # GABUNGIN SAP + FERTECON
    tambahan_row = df_fertecon[df_fertecon['BEDAT'] > df_all_m['BEDAT'].iloc[-1]].loc[:,['BEDAT','{}'.format(region_corr_fix)]].rename(columns={'{}'.format(region_corr_fix) : 'UNITPRICE'})
    final_data_concat = pd.concat([df_all_m,tambahan_row])
    # final_data_concat['EMATN'] = mtn
    final_data_concat['CORR_BENCHMARK'] = region_corr_fix
    df_all_m = final_data_concat

  df_all_m['EMATN'] = mtn

  return df_all_m

## RUN FUNCTION STEP 1

In [44]:
DAP_EMATN = [1000017,3000499, 3000539, 3000266, 3000508]
KCL_EMATN = [3000018,3000019, 3000072,3000048]
res_df_step1_alldap = pd.DataFrame()
res_df_step1_allkcl = pd.DataFrame()

for i in DAP_EMATN:
  res = olahData(i)
  res_df_step1_alldap = pd.concat([res_df_step1_alldap,res])

for i in KCL_EMATN:
  res = olahData(i)
  res_df_step1_allkcl = pd.concat([res_df_step1_allkcl,res])

# dapet 2 variable baru dari atas yaitu res_df_step1_alldap dan res_df_step1_allkcl

In [45]:
res_df_step1_allkcl

,BEDAT,UNITPRICE,EMATN
0,2016-02,278.000000,3000018
1,2016-04,278.000000,3000018
2,2016-05,186.249549,3000018
3,2016-06,94.512689,3000018
4,2016-07,278.000000,3000018
...,...,...,...
8,2020-04,262.100000,3000048
9,2020-07,232.200000,3000048
10,2020-09,230.200000,3000048
11,2020-11,232.200000,3000048


In [46]:
res_df_step1_alldap

,BEDAT,UNITPRICE,CORR_BENCHMARK,EMATN
0,2016-02,344.000000,DAP fob N. Africa_min,1000017
1,2016-03,339.600000,DAP fob N. Africa_min,1000017
2,2016-04,5.097802,DAP fob N. Africa_min,1000017
3,2016-06,160.408804,DAP fob N. Africa_min,1000017
4,2016-08,159.817321,DAP fob N. Africa_min,1000017
...,...,...,...,...
32,2021-09,664.000000,DAP cfr India_min,3000508
33,2021-10,674.000000,DAP cfr India_min,3000508
34,2021-11,796.000000,DAP cfr India_min,3000508
35,2021-12,799.000000,DAP cfr India_min,3000508


In [47]:
res_df_step1_alldapkcl = pd.concat([res_df_step1_alldap.loc[:,['BEDAT','UNITPRICE','EMATN']],res_df_step1_allkcl]).reset_index()
res_df_step1_alldapkcl

,index,BEDAT,UNITPRICE,EMATN
0,0,2016-02,344.000000,1000017
1,1,2016-03,339.600000,1000017
2,2,2016-04,5.097802,1000017
3,3,2016-06,160.408804,1000017
4,4,2016-08,159.817321,1000017
...,...,...,...,...
199,8,2020-04,262.100000,3000048
200,9,2020-07,232.200000,3000048
201,10,2020-09,230.200000,3000048
202,11,2020-11,232.200000,3000048


# STEP 2 - BUILD MODEL (ARIMA)

## FUNCTION

In [48]:
def split_data(df):
  #if not stationary then eliminate trend
  #Eliminate trend
  from pylab import rcParams
  rcParams['figure.figsize'] = 10, 6
  df_log = np.log(df.reset_index(drop=True))
  moving_avg = df_log.rolling(12).mean()
  std_dev = df_log.rolling(12).std()

  #split data into train and training set
  train_data, test_data = df_log[3:int(len(df_log)*0.75)], df_log[int(len(df_log)*0.75):]
  return train_data, test_data


def modelArima(df, ematn, train_data, test_data):
  #BuildModel
  model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
                        test='adf',       # use adftest to find optimal 'd'
                        max_p=3, max_q=3, # maximum p and q
                        m=1,              # frequency of series
                        d=None,           # let model determine 'd'
                        seasonal=False,   # No Seasonality
                        start_P=0, 
                        D=0, 
                        trace=True,
                        error_action='ignore',  
                        suppress_warnings=True, 
                        stepwise=True)
  # print(model_autoARIMA.summary())
  # model_autoARIMA.plot_diagnostics(figsize=(15,8))
  # plt.show()

  #Fitting
  model = sm.tsa.ARIMA(train_data, order=(1,1,2))  
  # model = sm.tsa.ARIMA(train_data, trend='c', enforce_stationarity=False, enforce_invertibility=False)  
  fitted = model.fit()  
  # print(fitted.summary())

  # Forecast
  fc, se, conf = fitted.forecast(3, alpha=0.05)  # 95% conf
  # Make as pandas series
  fc_series = pd.Series(fc, index=test_data.index)

  # report performance
  mse = mean_squared_error(test_data, fc_series)
  mae = mean_absolute_error(test_data, fc_series)
  rmse = math.sqrt(mean_squared_error(test_data, fc_series))
  mape = np.mean(np.abs(fc_series - test_data)/np.abs(test_data))

  #ADD NEW DATE PRED
  date_index = pd.date_range(df.tail(1).index.to_series().astype(str)[0], periods=len(fc_series)+1, freq='M').delete(0)
  df_pred_last = pd.DataFrame({"UNITPRICE": np.exp(fc_series).tolist()},index=date_index)
  df_pred_last = df_pred_last.reset_index()
  df_pred_last['index'] = pd.to_datetime(df_pred_last['index']).dt.to_period('M')
  df_pred_last = df_pred_last.set_index('index')

  #FINAL DF
  final_gab_last = pd.DataFrame()
  final_gab_last = pd.concat([df,df_pred_last])
  final_gab_last['MSE'] = mse
  final_gab_last['EMATN'] = ematn
  
  return final_gab_last

## CORETAN - INI CELL BUAT TEST SATU PER SATU EMATN

In [ ]:
## INI CELL BUAT TEST SATU SATU EMATN

DAP_EMATN = [1000017,3000499, 3000539, 3000266, 3000508]
KCL_EMATN = [3000018,3000019, 3000072,3000048]


# tes ngeset bedat sbg index
res_df_step1_alldap_perematn = res_df_step1_alldapkcl.set_index('BEDAT')
res_df_step1_alldap_perematn = pd.DataFrame(res_df_step1_alldap_perematn[res_df_step1_alldap_perematn['EMATN'] == 3000499]['UNITPRICE'])

# tes run function
train_data_o,test_data_o = split_data(res_df_step1_alldap_perematn)
final_df = modelArima(res_df_step1_alldap_perematn, 1000017, train_data_o, test_data_o)
final_df

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-16.392, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-16.283, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-18.913, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-12.663, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-17.128, Time=0.12 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-18.380, Time=0.11 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-16.692, Time=0.05 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0] intercept
Total fit time: 0.703 seconds


,UNITPRICE,MSE,EMATN
2019-03,314.840000,0.017499,1000017
2020-02,377.750000,0.017499,1000017
2020-05,316.400000,0.017499,1000017
2020-07,246.129151,0.017499,1000017
2020-08,310.830000,0.017499,1000017
2020-09,368.200000,0.017499,1000017
2020-10,387.000000,0.017499,1000017
2020-11,326.000000,0.017499,1000017
2020-12,324.000000,0.017499,1000017
2021-01,382.000000,0.017499,1000017


## RUN FUNCTION STEP 2 - UNTUK TES KESELURUHAN EMATN YANG DAPAT DI RUN DENGAN MODEL

In [49]:
DAP_EMATN = [1000017,3000499]
KCL_EMATN = [3000018,3000019]

GAB_EMATN = [1000017,3000499,3000018,3000019]

def predict_function(res_df_step1_alldapkcl,GAB_EMATN):
  res_predict = pd.DataFrame()
  for i in GAB_EMATN:
    # ngeset bedat sbg index
    res_df_step1_alldapkcl_perematn = res_df_step1_alldapkcl.set_index('BEDAT')
    res_df_step1_alldapkcl_perematn = pd.DataFrame(res_df_step1_alldapkcl_perematn[res_df_step1_alldapkcl_perematn['EMATN'] == i]['UNITPRICE'])

    # run function
    train_data_o,test_data_o = split_data(res_df_step1_alldapkcl_perematn)
    final_df = modelArima(res_df_step1_alldapkcl_perematn, i, train_data_o, test_data_o)

    res_predict = pd.concat([res_predict,final_df])

  return res_predict

In [50]:
# RESULTTTTTTTTTTTTTTTTTTTTTTT
res_gab_predict = predict_function(res_df_step1_alldapkcl,GAB_EMATN)
res_gab_predict

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=104.938, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=103.781, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=102.953, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.39 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.736 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-16.392, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-16.283, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-18.913, Time=0.15 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-12.663, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-17.128, Time=0.38 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-18.380, Time=0.26 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.69 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-16.692, Time=0.12 sec

Bes

,UNITPRICE,MSE,EMATN
2016-02,344.000000,0.312651,1000017
2016-03,339.600000,0.312651,1000017
2016-04,5.097802,0.312651,1000017
2016-06,160.408804,0.312651,1000017
2016-08,159.817321,0.312651,1000017
...,...,...,...
2022-01,319.569983,7.933110,3000019
2022-02,319.569983,7.933110,3000019
2022-03,319.569983,7.933110,3000019
2022-04,319.569983,7.933110,3000019


In [51]:
res_gab_predict['UNITPRICE'] = round(res_gab_predict['UNITPRICE'])

In [52]:
res_gab_predict

,UNITPRICE,MSE,EMATN
2016-02,344.0,0.312651,1000017
2016-03,340.0,0.312651,1000017
2016-04,5.0,0.312651,1000017
2016-06,160.0,0.312651,1000017
2016-08,160.0,0.312651,1000017
...,...,...,...
2022-01,320.0,7.933110,3000019
2022-02,320.0,7.933110,3000019
2022-03,320.0,7.933110,3000019
2022-04,320.0,7.933110,3000019


## UBAH RESULT KE RUPIAH

In [32]:
df_curr = pd.read_csv('Data Historis USD_IDR.csv')
# df_curr = df_curr.rename(columns={'Tanggal' : 'BEDAT'})
df_curr['Tanggal'] = pd.to_datetime(df_curr['Tanggal'])
df_curr['Terakhir'] = df_curr['Terakhir'].str[:-2].str.replace('.','').astype(float)

df_curr_m = df_curr.set_index('Tanggal').resample('M').mean().to_period('M')
df_curr_m = round(df_curr_m)

res_gab_predict_idr = res_gab_predict.copy().reset_index()
df_curr_m = df_curr_m.reset_index()

for i in range(len(res_gab_predict_idr['index'])):
    for j in range(len(df_curr_m['Tanggal'])):
        if (res_gab_predict_idr.loc[i,'index'] == df_curr_m.loc[j,'Tanggal']):
            usd_to_idr = res_gab_predict_idr.loc[i,'UNITPRICE']*df_curr.loc[j,'Terakhir']
            res_gab_predict_idr.loc[i, 'UNITPRICE'] = usd_to_idr

res_gab_predict_idr

,index,UNITPRICE,MSE,EMATN
0,2016-02,4986968.0,0.312651,1000017
1,2016-03,4902800.0,0.312651,1000017
2,2016-04,72050.0,0.312651,1000017
3,2016-06,2296960.0,0.312651,1000017
4,2016-08,2296800.0,0.312651,1000017
...,...,...,...,...
189,2022-01,4572800.0,7.933110,3000019
190,2022-02,4580800.0,7.933110,3000019
191,2022-03,4576000.0,7.933110,3000019
192,2022-04,4577600.0,7.933110,3000019


In [33]:
res_gab_predict_idr['UNITPRICE'].describe()

count    1.940000e+02
mean     4.523733e+06
std      2.956574e+06
min      2.630000e+02
25%      3.438088e+06
50%      4.528320e+06
75%      5.387542e+06
max      2.118756e+07
Name: UNITPRICE, dtype: float64

In [ ]:
res_gab_predict_idr = res_gab_predict_idr.rename(columns={'index' : 'Tanggal'})

# EXPORT

In [57]:
res_gab_predict = res_gab_predict.reset_index().rename(columns={'index' : 'Tanggal'})
res_gab_predict.to_csv('RESULTARIMAUSD_150522.csv')